# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

P. Mollière  ->  P. Mollière  |  ['P. Mollière']
J. Liu  ->  J. Liu  |  ['J. Liu']
N. Mackensen  ->  N. Mackensen  |  ['N. Mackensen']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']
K. Lee  ->  K. Lee  |  ['K. Lee']
Arxiv has 69 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2502.18559


extracting tarball to tmp_2502.18559...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2502.18559/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'model_based_Lbol_BC_6to13um.tex' from 'tmp_2502.18559/model_based_Lbol_BC_6to13um.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'model_based_Lbol_BC_1to5um.tex' from 'tmp_2502.18559/model_based_Lbol_BC_1to5um.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/op

...

extracting tarball to tmp_2502.18900... done.
Retrieving document from  https://arxiv.org/e-print/2502.19189


extracting tarball to tmp_2502.19189...

 done.
Retrieving document from  https://arxiv.org/e-print/2502.19224


extracting tarball to tmp_2502.19224...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.18559-b31b1b.svg)](https://arxiv.org/abs/2502.18559) | **ELemental abundances of Planets and brown dwarfs Imaged around Stars (ELPIS): II. The Jupiter-like Inhomogeneous Atmosphere of the First Directly Imaged Planetary-Mass Companion 2MASS 1207 b**  |
|| Z. Zhang, <mark>P. Mollière</mark>, J. J. Fortney, M. S. Marley |
|*Appeared on*| *2025-02-27*|
|*Comments*| *51 pages, 22 figures, and 8 tables. Submitted to AJ and fully revised following the referee report. This work is dedicated to the 20th anniversary of the discovery of 2MASS 1207 b, the first directly imaged planetary-mass companion. A review of 20 years of literature on the 2MASS 1207 A+b system is presented in Section 2*|
|**Abstract**|            2MASS 1207 b, the first directly imaged planetary-mass companion, has been instrumental in advancing our understanding of exoplanets and brown dwarfs over the past 20 years. We have performed extensive atmospheric retrieval analyses of 2MASS 1207 b's JWST/NIRSpec spectrum using petitRADTRANS and a new atmospheric inhomogeneity framework, which characterizes homogeneous atmospheres, patchy clouds, cloud-free hot spots, or the combination of patchy clouds and spots. Among 24 retrieval runs with various assumptions, the most statistically preferred model corresponds to the patchy cloud scheme, with Teff$=1174^{+4}_{-3}$ K, log(g)=$3.62^{+0.03}_{-0.02}$ dex, and R$=1.399^{+0.008}_{-0.010}$ R$_{\rm Jup}$, along with near-solar atmospheric compositions of [M/H]$=-0.05\pm0.03$ dex and C/O$=0.440\pm0.012$. This model suggests ~9% of 2MASS 1207 b's atmosphere is covered by thin iron clouds, producing L-dwarf-like spectra, while the remaining 91% consists of thick silicate and iron clouds, emitting blackbody-like spectra. These thin-cloud and thick-cloud regions resemble Jupiter's belts and zones, respectively; this scenario is consistently supported by other retrieval runs incorporating inhomogeneous atmospheres. We demonstrate that 2MASS 1207 b's weak CO absorption can be explained by the veiling effects of patchy thick clouds; the absence of 3.3um CH4 absorption is attributed to its hot thermal structure, which naturally leads to a CO-dominant, CH4-deficient atmosphere. The retrieved atmospheric models also match the observed variability amplitudes of 2MASS 1207 b. Our analysis reveals that the inferred atmospheric properties show significant scatter in less statistically preferred retrieval runs but converge to consistent values among the preferred ones. This underscores the importance of exploring diverse assumptions in retrievals to avoid biased interpretations of atmospheres and formation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.18567-b31b1b.svg)](https://arxiv.org/abs/2502.18567) | **"Frog-eyes" in Astronomy: Monitoring Binary Radial Velocity Variations Through A Pair of Narrow-Band Filters**  |
|| C. Zheng, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-02-27*|
|*Comments*| *17 pages, 11 figures, and 3 tables. Accepted for publication by SCPMA*|
|**Abstract**|            Spectroscopic observations are a crucial step in driving major discoveries in the era of time-domain surveys. However, the pace of current spectroscopic surveys is increasingly unable to meet the demands of rapidly advancing large-scale time-domain surveys. To address this issue, we propose the ``Frog-eyes" system, which employs a pair of narrow-band filters: one positioned near a strong absorption line to capture signals from Doppler shifts, and the other placed on the adjacent continuum to monitor intrinsic variations. The combination of observations from the two filters enables the extraction of radial velocity (RV) curves from a large sample of binary stars, and is particularly efficient for single-lined binaries (SB1), using photometric techniques. Comprehensive mock simulations on SB1 demonstrate that the binary orbital parameters can be precisely measured from the extracted RV curves for binary systems where the primary star has an effective temperature greater than 6000 K. With a typical ground-based photometric precision of approximately 0.3%, the uncertainties in the derived semi-amplitude K and eccentricity e are less than 10% and 0.1, respectively, for binary systems with K $\ge$ 30 km/s. These encouraging results are further validated by real observations of the hot subdwarf-white dwarf binary system HD 265435, using a non-specialized ``Frog-eyes" system installed on the Chinese 2.16m telescope. Once this system is properly installed on large-field-of-view survey telescopes, the rate of acquiring RV curves for binaries will approach their detection rate in leading time-domain photometric surveys.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.18824-b31b1b.svg)](https://arxiv.org/abs/2502.18824) | **Galactic-scale emission-line outflow from the radio-loud quasar 3C 191**  |
|| Q. Zhao, et al. -- incl., <mark>M. Sharma</mark> |
|*Appeared on*| *2025-02-27*|
|*Comments*| *18gapes, 10 figures, Accepted for publication in ApJS*|
|**Abstract**|            Quasar feedback is routinely invoked as an indispensable ingredient in galaxy formation models. Galactic outflows are a crucial agent of quasar feedback that frequently manifest themselves in absorption and emission lines. Measuring the size and energetics of outflows based on absorption lines remains a challenge, and integral-field spectroscopy (IFS) mapping in emission lines is complementary. We present a VLT/SINFONI IFS mapping of quasar 3C 191 at $z \sim 2$, in which the outflow has been analyzed in absorption line spectroscopy. Three components are found based on the morphology and kinetics of [OIII]-emitting gas: a unshifted component which consistent with the systemic redshift and the location of the nucleus, a blueshifted in the north, and a redshifted in the south. The latter two components have velocities $\sim$ 600 km s$^{-1}$ and projected extents of 5 and 11 kpc, respectively, suggesting a biconical outflow structure. The blueshifted component's velocity is consistent with that derived from absorption lines. Using the electron density measured by the absorption lines and the luminosity and velocity of [OIII] outflow, we derive the mass outflow rate to be $\dot{M} \sim $ 9.5-13.4 M$_\odot$ yr$^{-1}$ and kinetic luminosity $\dot{E}_{\rm kin}$ ~ 2.5-3.7 $\times 10^{42}$ erg s$^{-1}$, consistent with absorption line analyses with VLT/Xshooter spectrum. The kinetic luminosity is only 0.01% of the bolometric luminosity, rendering a relatively weak outflow compared to typical expectation for effective feedback.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.18825-b31b1b.svg)](https://arxiv.org/abs/2502.18825) | **The distance of quasar outflows from the central source: The first consistent values from emission and absorption determinations**  |
|| <mark>M. Sharma</mark>, et al. |
|*Appeared on*| *2025-02-27*|
|*Comments*| *18 pages, 11 figures. Accepted for publication in ApJ along with the companion paper*|
|**Abstract**|            Measuring the distance of quasar outflows from the central source ($R$) is essential for determining their importance for AGN feedback. There are two methods to measure $R$: 1) A direct determination using spatially resolved Integral Field Spectroscopy (IFS) of the outflow in emission. 2) An indirect method which uses the absorption troughs from ionic excited states. The column density ratio between the excited and resonance states yields the outflow number density. Combined with a knowledge of the outflow's ionization parameter, $R$ can be determined. Generally, the IFS method probes $R$ range of several kpc or more, while the absorption method usually yields $R$ values of less than 1 kpc. There is no inconsistency between the two methods as the determinations come from different objects. Here we report the results of applying both methods to the same quasar outflow, where we derive consistent determinations of $R$ $\approx$ 5 kpc. This is the first time where the indirect absorption $R$ determination is verified by a direct spatially resolved IFS observation. In addition, the velocities (and energetics) from the IFS and absorption data are also found to be consistent. Therefore, these are two manifestations of the same outflow. In this paper we concentrate on the absorption $R$ determination for the outflow seen in quasar 3C 191 using VLT/X-shooter observations. We also reanalyze an older absorption determination for the outflow based on Keck/HIRES data and find that revised measurement to be consistent with ours. Our companion paper details the IFS analysis of the same object.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.18900-b31b1b.svg)](https://arxiv.org/abs/2502.18900) | **Detection of [C\,{\sc i}] Emission in Nebular Spectra of a Peculiar Type Ia Supernova 2022pul**  |
|| <mark>J. Liu</mark>, et al. |
|*Appeared on*| *2025-02-27*|
|*Comments*| *11 pages, 6 figures, 2 tables, accepted for publication in ApJL*|
|**Abstract**|            SN~2022pul gains special attention due to its possible origin of a super-Chandarsekhar-mass white dwarf explosion (or called a 03fg-like type Ia supernova), which shows prominent [O\,{\sc i}], [Ne\,{\sc i}], and [Ca\,{\sc ii}] lines in its late-time spectra taken at $\sim+$300 days after the peak brightness. In this paper, we present new optical observations for this peculiar object, extending up to over 500 days after the peak brightness. In particular, in the $t\approx+515$ days spectrum, we identified for the first time the presence of narrow emission from [C\,{\sc i}] $\lambda\lambda9824, 9850$, which appears asymmetric and quite similar to the accompanied [O\,{\sc i}] $\lambda6300$ line in strength and profile. Based on the violent merger model that accounts well for previous observations but leaves little carbon in the center of the ejecta, this carbon line can be reproduced by increasing the degree of clumping in the ejecta and setting the carbon mass the same as that of oxygen ($\sim$0.06 $M_{\odot}$) in the innermost region ($\lesssim 2000$ km s$^{-1}$). In principle, the central carbon could come from the secondary white dwarf (WD) if it is ignited when hit by the shockwave of the explosion of the primary WD and explodes as a Ca-rich supernova, whereas pure deflagration of a super-Chandarsekhar-mass WD can account for such unburnt carbon more naturally.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.19189-b31b1b.svg)](https://arxiv.org/abs/2502.19189) | **Fermi-detection of gamma-ray Emissions from the Hot Coronae of Radio-quiet Active Galactic Nuclei**  |
|| F.-L. Collaboration, J.-M. Wang, <mark>J. Li</mark>, J.-R. Liu |
|*Appeared on*| *2025-02-27*|
|*Comments*| *Submitted*|
|**Abstract**|            Relativistic jets around supermassive black holes (SMBHs) are well-known powerful $\gamma$-ray emitters. In absence of the jets in radio-quiet active galactic nuclei (AGNs), how the SMBHs work in $\gamma$-ray bands is still unknown despite of great observational efforts made in the last 3 decades. Considering the previous efforts, we carefully select an AGN sample composed of 37 nearby Seyfert galaxies with ultra-hard X-rays for the goals of $\gamma$-ray detections by excluding all potential contamination in this band. Adopting a stacking technique, here we report the significant $\gamma$-ray detection (${\rm TS}=30.6$, or $5.2\,\sigma$) from the sample using 15-year Fermi-Large Area Telescope (LAT) observation. We find an average $\gamma$-ray luminosity of the sample as $(1.5\pm1.0)\times10^{40}{\,\rm erg\,s^{-1}}$ at energies from 1-300\,GeV. Limited by the well-known pair production from the interaction of $\gamma$-rays with low energy photons, $\gtrsim$ several GeV $\gamma$-rays are found to originate from an extended corona ($\sim 2.7\times 10^6\,R_{\rm g}$), whereas the canonical much more compact X-ray corona ($\sim 10\,R_{\rm g}$) is responsible for 1 to several GeV $\gamma$-rays. The finding of the compact region lends to strong supports to the long-time theoretical expectations, but the extended corona is beyond all the existing models. One promising scenario is that the electron-positron pairs produced in the compact X-ray corona would expand as fireball, similar to that in $\gamma$-ray bursts, forming the structure of extended corona.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.19224-b31b1b.svg)](https://arxiv.org/abs/2502.19224) | **Coronal Abundance Fractionation Linked to Chromospheric Transverse MHD Waves in a Solar Active Region Observed with FISS/GST and EIS/Hinode**  |
|| K.-S. Lee, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2025-02-27*|
|*Comments*| *13 pages, 10 figures, submitted to A&A*|
|**Abstract**|            Elemental abundances in the solar corona differ from those in the photosphere, with low first ionization potential (FIP) elements being enhanced, a phenomenon known as the FIP effect. This enhancement is attributed to ponderomotive forces linked to magnetohydrodynamic (MHD) waves, particularly incompressible transverse waves. Our study investigates the relationship between coronal abundance fractionation and chromospheric transverse MHD waves by examining the spatial correlation between FIP fractionation and these waves and by analyzing their properties to test the ponderomotive force model. We used H alpha data from the Fast Imaging Solar Spectrograph at the Goode Solar Telescope to detect chromospheric transverse MHD waves and \ion{Si}{X} (low FIP) and \ion{S}{X} (high FIP) spectra from Hinode EUV Imaging Spectrometer to determine relative abundances in an active region. Extrapolated linear force free magnetic fields from Solar Dynamics Observatory/Helioseismic and Magnetic Imager magnetograms further linked the observed chromospheric waves with coronal composition. Approximately 400 wave packets were identified and characterized by their period, velocity amplitude, propagation speed, and direction. These incompressible or weakly compressible waves were mainly observed near loop footpoints in the sunspot penumbra and superpenumbral fibrils. Regions of high FIP fractionation coincided with closed magnetic fields where these waves were present, and low-frequency, downward-propagating waves comprised about 43/% of the total. Our results demonstrate a strong correlation between coronal abundance fractionation and chromospheric transverse MHD waves, supporting the view that the FIP effect is driven by the ponderomotive force from these waves.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.18651-b31b1b.svg)](https://arxiv.org/abs/2502.18651) | **Variability of Central Stars of Planetary Nebulae with the Zwicky Transient Facility. II. Long-Timescale Variables including Wide Binary and Late Thermal Pulse Candidates**  |
|| S. Bhattacharjee, et al. -- incl., <mark>N. Mackensen</mark>, <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-02-27*|
|*Comments*| *23 pages (including appendix), 20 figures, 2 tables; Submitted to PASP, Comments are Welcome!*|
|**Abstract**|            In this second paper on the variability survey of central stars of planetary nebulae (CSPNe) using ZTF, we focus on the 11 long-timescale variables with variability timescales ranging from months to years. We also present preliminary analyses based on spectroscopic and/or photometric follow-up observations for six of them. Among them is NGC 6833, which shows a 980 day periodic variability with strange characteristics: 'triangle-shaped' brightening in $r$, $i$, and WISE bands but almost coincidental shallow dips in the $g$-band. We speculate this to be a wide but eccentric binary with the same orbital period. Long-period near-sinusoidal variability was detected in two other systems, NGC 6905 and Kn 26, with periods of 700 days and 230 days, respectively, making them additional wide-binary candidates. The latter also shows a short period at 1.18 hours which can either be from a close inner binary or pulsational origin. We present CTSS 2 and PN K 3-5 which show brightening and significant reddening over the whole ZTF baseline. A stellar model fit to the optical spectrum of CTSS 2 reveals it to be one of the youngest post-AGB CSPN known. Both show high-density emission-line cores. These appear to be late thermal pulse candidates, currently evolving towards the AGB phase, though alternative explanations are possible. We then present recent HST/COS ultraviolet spectroscopy of the known wide-binary candidate LoTr 1 showing that the hot star is a spectroscopic twin of the extremely hot white dwarf in UCAC2 46706450. We think that the long photometric period of 11 years is the binary orbital period. Finally, we briefly discuss the ZTF light curves of the remaining variables, namely Tan 2, K 3-20, WHTZ 3, Kn J1857+3931, and IPHAS J1927+0814. With these examples, we present the effectiveness of the von Neumann statistics and Pearson Skew-based metric space in searching for long-timescale variables.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2502.18559/./ceq_2M1207b_patchy_FeSifor1_FeSifor2_evo_data_vs_models_combo.png', 'tmp_2502.18559/./inhomogeneity_framework.png', 'tmp_2502.18559/./ch4onset_Teff_effects.png', 'tmp_2502.18559/./ch4onset_MH_effects.png']
copying  tmp_2502.18559/./ceq_2M1207b_patchy_FeSifor1_FeSifor2_evo_data_vs_models_combo.png to _build/html/
copying  tmp_2502.18559/./inhomogeneity_framework.png to _build/html/
copying  tmp_2502.18559/./ch4onset_Teff_effects.png to _build/html/
copying  tmp_2502.18559/./ch4onset_MH_effects.png to _build/html/
exported in  _build/html/2502.18559.md
    + _build/html/tmp_2502.18559/./ceq_2M1207b_patchy_FeSifor1_FeSifor2_evo_data_vs_models_combo.png
    + _build/html/tmp_2502.18559/./inhomogeneity_framework.png
    + _build/html/tmp_2502.18559/./ch4onset_Teff_effects.png
    + _build/html/tmp_2502.18559/./ch4onset_MH_effects.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\dotdeg}{\rlap{.}^\circ}$
$\newcommand{\dotarcsec}{\rlap{.}"}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# ELemental abundances of Planets and brown dwarfs Imaged around Stars (ELPIS): \\II. The Jupiter-like Inhomogeneous Atmosphere of the First Directly Imaged Planetary-Mass Companion 2MASS 1207 b

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.18559-b31b1b.svg)](https://arxiv.org/abs/2502.18559)<mark>Appeared on: 2025-02-27</mark> -  _51 pages, 22 figures, and 8 tables. Submitted to AJ and fully revised following the referee report. This work is dedicated to the 20th anniversary of the discovery of 2MASS 1207 b, the first directly imaged planetary-mass companion. A review of 20 years of literature on the 2MASS 1207 A+b system is presented in Section 2_

</div>
<div id="authors">

Z. Zhang (张周健), <mark>P. Mollière</mark>, J. J. Fortney, M. S. Marley

</div>
<div id="abstract">

**Abstract:** 2MASS 1207 b, the first directly imaged planetary-mass companion, has been instrumental in advancing our understanding of exoplanets and brown dwarfs over the past 20 years. We have performed extensive atmospheric retrieval analyses of 2MASS 1207 b's JWST/NIRSpec spectrum using \texttt{petitRADTRANS} and a new atmospheric inhomogeneity framework, which characterizes homogeneous atmospheres, patchy clouds, cloud-free hot spots, or the combination of patchy clouds and spots. Among 24 retrieval runs with various assumptions, the most statistically preferred model corresponds to the patchy cloud scheme, with $T_{\rm eff}=1174^{+4}_{-3}$ K, $\log{(g)}=3.62^{+0.03}_{-0.02}$ dex, and $R=1.399^{+0.008}_{-0.010}$ R $_{\rm Jup}$ , along with near-solar atmospheric compositions of [ M/H ] $=-0.05\pm0.03$ dex and C/O $=0.440\pm0.012$ . This model suggests $\sim9\%$ of 2MASS 1207 b's atmosphere is covered by thin iron clouds, producing L-dwarf-like spectra, while the remaining $91\%$ consists of thick silicate and iron clouds, emitting blackbody-like spectra. These thin-cloud patches and thick-cloud regions resemble Jupiter's belts and zones, respectively, and this scenario is consistently supported by other retrieval runs incorporating inhomogeneous atmospheres. We demonstrate that the weak CO absorption of 2MASS 1207 b can be explained by the veiling effects of patchy thick clouds; the absence of 3.3 $\mu$ m CH $_{4}$ absorption is attributed to its hot thermal structure, which naturally leads to a CO-dominant, CH $_{4}$ -deficient atmosphere. The retrieved atmospheric models also match the observed variability amplitudes of 2MASS 1207 b. Our analysis reveals that the inferred atmospheric properties show significant scatter in less statistically preferred retrieval runs but converge to consistent values among the preferred ones. This underscores the importance of exploring diverse assumptions in retrievals to avoid biased interpretations of $\new{atmospheric properties}$ and formation $\new{pathways}$ .

</div>

<div id="div_fig1">

<img src="tmp_2502.18559/./ceq_2M1207b_patchy_FeSifor1_FeSifor2_evo_data_vs_models_combo.png" alt="Fig6" width="100%"/>

**Figure 6. -** Results of our most preferred retrieval setup, \texttt{QEQ-1}(see Tables \ref{tab:retrieved_qeq_key}--\ref{tab:retrieved_qeq_cloud} and discussions in Section \ref{subsec:inhomo}), which has the lowest BIC value. The top panel compares the observed JWST/NIRSpec spectrum of 2MASS 1207 b (black circles) with the best-fit spectrum from \texttt{QEQ-1}(orange). The emergent spectra from each atmospheric column are plotted in purple and green, with fluxes scaled by their corresponding coverage fractions. The blue error bars ($\sigma$) represent the total error budget, where $\sigma = \sqrt{\sigma_{\rm obs}^{2} + b_{\rm chunk}^{2}}$, with $\sigma_{\rm obs}$ as the observed flux uncertainty and $b_{\rm chunk}$ as the fitted hyper-parameter (explained in Section \ref{subsec:params_priors}). The second panel from the top shows the residual spectrum between the observed and best-fit spectra, scaled by $\sigma$. The middle-left panel presents the confidence intervals for the retrieved T-P profiles. The remaining panels present the posteriors of key parameters, with median values and confidence intervals summarized in Table \ref{tab:retrieved_qeq_key}.  (*fig:patchy_fesifor_evo*)

</div>
<div id="div_fig2">

<img src="tmp_2502.18559/./inhomogeneity_framework.png" alt="Fig5" width="100%"/>

**Figure 5. -** Four schemes in our atmospheric inhomogeneity framework (Section \ref{subsec:inhomo_framework}).  (*fig:inhomo*)

</div>
<div id="div_fig3">

<img src="tmp_2502.18559/./ch4onset_Teff_effects.png" alt="Fig11.1" width="50%"/><img src="tmp_2502.18559/./ch4onset_MH_effects.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** _ Top_: The left panel shows T-P profiles from the \texttt{Sonroa Diamondback} models with varying effective temperatures: $T_{\rm eff} = 1700$ K (red), $1500$ K (orange), 1300 K (blue), and 1100 K (purple). All models have $\log{(g)} = 4.5$ dex, [M/H]$=0$ dex, and $f_{\rm sed} = 2$. The $3-4$ $\mu$m photosphere of each model is highlighted with a thick line. The CO$-$CH$_{4}$ transition curve (see Figure \ref{fig:tp_vs_f20}) is shown as a dashed line. The right panel presents the corresponding spectra, with CH$_{4}$ absorption features (R, Q, and P branches) labeled.  _ Bottom_: These two panels follow the same format as the top panels but present the T-P profiles and spectra for models with different [M/H] values: $+0.5$ dex (red), $0$ dex (orange), and $-0.5$ dex (blue). All models have $T_{\rm eff} = 1400$ K, $\log{(g)} = 4.5$ dex, and $f_{\rm sed} = 2$.  (*fig:onset_ch4_teffmh*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.18559"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

343  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
